Imported Libaries

In [1]:
import os
root_dir = r'c:\Users\jf79\Cloned Repos\Footfall-Repo'
os.chdir(root_dir)

import pandas as pd

import myfuncs.myfuncs as mf
import myfuncs.footfallfuncs as ff

mf.read_directory()

Your Current Directory is: c:\Users\jf79\Cloned Repos\Footfall-Repo
Files in: ['.git', 'Footfall.venv', 'Footfall_Scripts', 'myfuncs', 'README.md']



In [2]:
cwd = r'C:/Users/jf79/OneDrive - Office Shared Service/Documents/H&F Analysis/Footfall and Spend Analysis/Footfall Data'
mf.read_directory(cwd)

Directory being read is: C:/Users/jf79/OneDrive - Office Shared Service/Documents/H&F Analysis/Footfall and Spend Analysis/Footfall Data
Files in: ['Area Based', 'Hex Based', 'LSOA Based', 'Mapping']



In [3]:
footfall_2024_data = pd.read_csv(f'{cwd}/Hex Based/Footfall Counts/hex_3hourly_counts_2024.csv')
hex_to_borough_data = pd.read_csv(f'{cwd}/Hex Based/Footfall Counts/BT Hex ID to Borough Lookup Table.csv')
relevant_hexes_data = pd.read_csv(f'{cwd}/Hex Based/Relevant Hexes/Relevant Hexes.csv')
bespoke_areas_data = pd.read_csv(f'{cwd}/Area Based/bespoke_3hourly_counts_2022-05-01_2025-02-09.csv')

In [4]:
values_to_keep = [
    "Sands End","Fulham Town","Parsons Green & Sandford","Palace & Hurlingham","Munster",
    "Walham Green","Lillie","Fulham Reach","West Kensington","Hammersmith Broadway",
    "Avonmore","Ravenscourt","Brook Green","Grove","Addison","Wendell Park",
    "Coningham","Shepherd's Bush Green","Wormholt","White City","College Park & Old Oak"
]

bespoke_areas_data = pd.read_csv(f'{cwd}/Area Based/bespoke_3hourly_counts_2022-05-01_2025-02-09.csv')
ward_footfall = bespoke_areas_data.copy()

ward_footfall = ward_footfall[ward_footfall['name'].isin(values_to_keep)]
ward_footfall['hours'] = ward_footfall['hours'].str.replace("'","")
ward_footfall.rename(
    columns={
        'hours':'time_indicator',
    },
    inplace=True
)
ward_footfall

,bespoke_area_id,name,count_date,time_indicator,resident,visitor,worker,ave_loyalty_percentage,ave_dwell_time
1953166,257,Sands End,2022-05-01,00-03,957,179,89,20.26,109.56
1953167,257,Sands End,2022-05-01,03-06,12538,4633,969,2.16,104.36
1953168,257,Sands End,2022-05-01,06-09,13383,5149,1224,3.42,139.48
1953169,257,Sands End,2022-05-01,09-12,15997,8379,1660,6.97,100.04
1953170,257,Sands End,2022-05-01,12-15,14451,10695,1972,5.95,88.25
...,...,...,...,...,...,...,...,...,...
2122626,277,College Park & Old Oak,2025-02-09,09-12,31155,13968,5345,7.69,87.92
2122627,277,College Park & Old Oak,2025-02-09,12-15,30906,21957,5956,9.36,73.75
2122628,277,College Park & Old Oak,2025-02-09,15-18,30295,23090,6002,8.35,74.16
2122629,277,College Park & Old Oak,2025-02-09,18-21,31449,17293,4932,6.38,83.96


In [5]:
footfall_2024 = footfall_2024_data.copy()
relevant_hexes = relevant_hexes_data.copy()

footfall_2024 = pd.merge(
    relevant_hexes_data,
    footfall_2024,
    left_on='Hex_ID',
    right_on='hex_id',
    how='left'
)

In [6]:
typical_day_averages = ff.typical_footfall(
    ward_footfall, '2024-01-01', '2024-12-31',
    footfall_type=['residents','workers','visitors'],
    day_night=True,
    primary_key='name'
)
for i in range(len(typical_day_averages)):
    if 'averages' in typical_day_averages[i].columns.to_list():
        display(typical_day_averages[i].sort_values(by='averages', ascending=False))
    elif 'daytime_mean' in typical_day_averages[i].columns.to_list():
        display(typical_day_averages[i].sort_values(by='daytime_mean', ascending=False))
    else:
        display(typical_day_averages[i])

Calculating typical daily footfall...
For Weedays and Weekends and Weekly averages...



c:\Users\jf79\Cloned Repos\Footfall-Repo\myfuncs\footfallfuncs.py:231: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  footfall_data.loc[:, columns_to_fill] = footfall_data[columns_to_fill].applymap(lambda x: np.nan if x < 0 else x)
c:\Users\jf79\Cloned Repos\Footfall-Repo\myfuncs\footfallfuncs.py:232: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  footfall_data[columns_to_fill] = footfall_data[columns_to_fill].fillna(0)



Aggregating footfall data...
Missing kwargs: {'day_night'}
These args will be set to default values

Applying features...
Features applied.

Error aggregating footfall data: "Column(s) ['dwell_time', 'loyalty_percentage'] do not exist"


Transforming to daynight...
Error transforming to daynight: 'corrected_value_total'



KeyError: 'year'

In [ ]:
# mf.export_to_csv(typical_day_averages[0])